In [144]:
git = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"

In [145]:
import pandas as pd

In [146]:
url = f'{git}?raw=1'
df = pd.read_csv(url)

In [147]:
df = df.iloc[:300]

In [148]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [8]:
model_name = "multi-qa-mpnet-base-dot-v1"

In [9]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/home/yusuf/.local/share/virtualenvs/llm-datatalks-club-OTlqznPF/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/yusuf/.local/share/virtualenvs/llm-datatalks-club-OTlqznPF/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
answer_llm = df.iloc[0].answer_llm

In [11]:
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

## Q1

In [14]:
embedding_model.encode(answer_llm)[0]

-0.42244658

## Q2

In [23]:
answer_llms = df.answer_llm
answer_origs = df.answer_orig

'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork'

In [25]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)
    
    return v_llm.dot(v_orig)


In [26]:
from tqdm.auto import tqdm

In [27]:
results = df.to_dict(orient='records')

In [108]:
evaluations = []

for record in tqdm(results):
    score = compute_similarity(record)
    evaluations.append(score)

100%|██████████| 300/300 [01:21<00:00,  3.68it/s]


In [86]:
df['evaluations'] = evaluations

In [87]:
df.evaluations.describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547925
25%       24.307845
50%       28.336873
75%       31.674312
max       39.476013
Name: evaluations, dtype: float64

## Q3

In [72]:
import numpy as np

In [109]:
def compute(records):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = embedding_model.encode(answer_llm)
    norm_llm = np.sqrt((v_llm * v_llm).sum())
    v_norm_llm = v_llm / norm_llm

    v_orig = embedding_model.encode(answer_orig)
    norm_orig = np.sqrt((v_orig * v_orig).sum())
    v_norm_orig = v_orig / norm_orig

    return v_norm_llm.dot(v_norm_orig)

In [110]:
cosine = []

for record in tqdm(results):
    score = compute(record)
    cosine.append(score)

100%|██████████| 300/300 [01:24<00:00,  3.55it/s]


In [112]:
df['cosine'] = cosine
df.cosine.describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine, dtype: float64

## Q4

In [113]:
from rouge import Rouge


In [149]:
df_subset = df[df["document"] == "5170565b"]

In [150]:
df_subset

,answer_llm,answer_orig,document,question,course
10,"Yes, all sessions are recorded, so if you miss...","Everything is recorded, so you won’t miss anyt...",5170565b,Are sessions recorded if I miss one?,machine-learning-zoomcamp
11,"Yes, you can ask your questions in advance if ...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I ask questions in advance if I can't atte...,machine-learning-zoomcamp
12,"If you miss a session, don't worry! Everything...","Everything is recorded, so you won’t miss anyt...",5170565b,How will my questions be addressed if I miss a...,machine-learning-zoomcamp
13,"Yes, there is a way to catch up on a missed se...","Everything is recorded, so you won’t miss anyt...",5170565b,Is there a way to catch up on a missed session?,machine-learning-zoomcamp
14,"Yes, you can still interact with instructors a...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I still interact with instructors after mi...,machine-learning-zoomcamp


In [312]:
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df_subset['answer_llm'], df_subset['answer_orig'])[0]

In [168]:
(df_subset['answer_llm'], df_subset['answer_orig'])[0]

10    Yes, all sessions are recorded, so if you miss...
11    Yes, you can ask your questions in advance if ...
12    If you miss a session, don't worry! Everything...
13    Yes, there is a way to catch up on a missed se...
14    Yes, you can still interact with instructors a...
Name: answer_llm, dtype: object

In [215]:
scores["rouge-1"]["f"]

0.45454544954545456

## Q5

In [164]:
(scores["rouge-1"]["f"] + scores["rouge-2"]["f"] + scores["rouge-l"]["f"]) / 3

0.35490034990035496

## Q6

In [217]:
scores

{}

In [189]:
def rouge(record):
    score = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])[0]
    return score


In [357]:
scores = {
    "rouge-1": {"r" : [], "p": [], "f":[]},
    "rouge-2": {"r" : [], "p": [], "f":[]},
    "rouge-l": {"r" : [], "p": [], "f":[]}
}

In [358]:
for record in tqdm(results):
    score = rouge(record)
    scores["rouge-1"]["f"].append(score["rouge-1"]["f"])
    scores["rouge-2"]["f"].append(score["rouge-2"]["f"])
    scores["rouge-l"]["f"].append(score["rouge-l"]["f"])
    scores["rouge-1"]["p"].append(score["rouge-1"]["p"])
    scores["rouge-2"]["p"].append(score["rouge-2"]["p"])
    scores["rouge-l"]["p"].append(score["rouge-l"]["p"])
    scores["rouge-1"]["r"].append(score["rouge-1"]["r"])
    scores["rouge-2"]["r"].append(score["rouge-2"]["r"])
    scores["rouge-l"]["r"].append(score["rouge-l"]["r"])

100%|██████████| 300/300 [00:00<00:00, 403.49it/s]


In [365]:
pd.DataFrame(scores["rouge-2"]["f"]).mean()

0    0.206965
dtype: float64

In [366]:
pd.DataFrame(scores["rouge-2"]["r"]).mean()

0    0.198613
dtype: float64

In [367]:
pd.DataFrame(scores["rouge-2"]["p"]).mean()

0    0.258626
dtype: float64

In [376]:
pd.DataFrame(pd.DataFrame(scores)["rouge-2"]["f"]).mean()

0    0.206965
dtype: float64